<a href="https://colab.research.google.com/github/echung2/echung2/blob/master/%ED%95%9C%EA%B5%AD%ED%98%84%EB%8C%80%EB%AC%B8%ED%95%99%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D%EC%97%B0%EA%B5%AC_7%EC%A3%BC%EC%B0%A8_%EA%B3%B5%EA%B8%B0%EC%96%B4_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 공기어 분석

### 0. 자료 준비

In [ ]:
# 나눔고딕
!apt-get update -qq
!apt-get install fonts-nanum* -qq

# 패키지 설치
!pip install kiwipiepy flashtext -q

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from kiwipiepy import Kiwi, Option
kiwi = Kiwi()
kiwi.prepare()

from tqdm.notebook import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'

import itertools
from collections import Counter

import regex #확장된 정규표현식. 일반 정규표현식은 import re

from flashtext import KeywordProcessor
kp = KeywordProcessor()

In [ ]:
# 이인직 소설 자료 다운로드
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1AY763FcPXN_iBo_sVMXHugQ8UHFvb_nN' -O lee.xlsx

In [ ]:
df = pd.read_excel('lee.xlsx')
df

### 1. 형태소 분석 및 전처리

##### 사용자 사전 등록

In [ ]:
kiwi.analyze('혈의누의 옥련은 운다. 훌쩍이다')

[([('혈', 'NNG', 0, 1),
   ('의', 'JKG', 1, 1),
   ('누', 'NNG', 2, 1),
   ('의', 'JKG', 3, 1),
   ('옥련', 'NNG', 5, 2),
   ('은', 'JX', 7, 1),
   ('울', 'VV', 8, 2),
   ('ᆫ다', 'EF', 10, 1),
   ('.', 'SF', 11, 1),
   ('훌쩍이', 'VV', 13, 3),
   ('다', 'EC', 16, 1)],
  -91.01134490966797)]

In [ ]:
kiwi.load_user_dictionary("dic.txt")
# kiwi.prepare()

In [ ]:
kiwi.analyze('옥련은 혈의누의 주인공이다.')

[([('옥련', 'NNG', 0, 2),
   ('은', 'JX', 2, 1),
   ('혈', 'NNG', 4, 1),
   ('의', 'JKG', 5, 1),
   ('누', 'NNG', 6, 1),
   ('의', 'JKG', 7, 1),
   ('주인공', 'NNG', 9, 3),
   ('이', 'VCP', 12, 1),
   ('다', 'EF', 13, 1),
   ('.', 'SF', 14, 1)],
  -57.77159881591797)]

##### 형태소 분석
품사 참고 : https://bab2min.github.io/kiwipiepy/v0.9.2/kr/#_7

In [ ]:
# 몇가지 품사 제외한 모든 품사 추출
def tokenize(sent):
    res, score = kiwi.analyze(sent)[0] # 첫번째 결과를 사용
    return [word + ('다' if tag.startswith('V') else '') # 동사/형용사에는 '다'를 붙여줌
    # return [word + ('다' if tag.startswith('V') else '')+ '/'+ tag # 동사/형용사에는 '다'를 붙여줌 + / 품사
            for word, tag, _, _ in res
            if not tag.startswith('E') and not tag.startswith('J') and not tag.startswith('S')] # 조사, 어미, 특수기호 및 stopwords에 포함된 단어는 제거

In [ ]:
# 몇가지 품사 제외한 모든 품사 추출 + 품사 태그 포함
def tokenize_tag(sent):
    res, score = kiwi.analyze(sent)[0] # 첫번째 결과를 사용
    return [word + ('다' if tag.startswith('V') else '')+ '/'+ tag # 동사/형용사에는 '다'를 붙여줌 + / 품사
            for word, tag, _, _ in res
            if not tag.startswith('E') and not tag.startswith('J') and not tag.startswith('S')] # 조사, 어미, 특수기호 및 stopwords에 포함된 단어는 제거

In [ ]:
# 특정 품사만 추출
def tokenize_part(sent):
    res, score = kiwi.analyze(sent)[0] # 첫번째 결과를 사용
    return [word
            for word, tag, _, _ in res
            if tag.startswith('NN')] # 'NN'으로 시작하는 품사만 추출 = 명사만 추출

In [ ]:
df['paragraph'][0]

In [ ]:
tokenize(df['paragraph'][0])

In [ ]:
df['tokens'] = df['paragraph'].progress_map(lambda x:tokenize(x))

In [ ]:
df['tokens']

##### 불용어 제거

In [ ]:
# 상위 n개 단어 확인
token_list = list(itertools.chain(*df['tokens'].tolist()))
cnt = Counter(token_list)
cnt.most_common(30) # 상위 20개

In [ ]:
 # 불용어 리스트
stopwords = ['이다','하다','그','이','들','하','위하','있다','것','없다','아니하다'] # 품사가 포함된 경우에는 '이다/VCP' 이런식으로 바꾸어야 함

In [ ]:
# 불용어 제거
df['tokens'] = df['tokens'].map(lambda x:[w for w in x if not w in set(stopwords)])

In [ ]:
# 유효한 1음절은 살리고 나머지는 제거
hangul_1_except = regex.compile(r'^(?!삶|앎|꿈|말|시)\p{Hangul}{1}$') # 파이프(|)로 구분해 입력. 맨마지막 단어에는 파이프 입력하지 말것.
df['tokens'] = df['tokens'].progress_map(lambda x:[w for w in x if not hangul_1_except.match(w)]) #매치되는 것을 제거한다.

In [ ]:
df['tokens']

##### 동의어 처리

In [ ]:
# flashtext용 동의어 딕셔너리
# 통일단어 : 바꿀단어
#'startup':['start up','start ups','start-up','start-ups','start_up','start_ups'],

synonym_dict = {
'옥련':['옥련이'] # 부인?

}
kp.add_keywords_from_dict(synonym_dict)

In [ ]:
# 동의어 처리 실행
df['tokens'] = [[kp.replace_keywords(x) for x in w] for w in tqdm(df['tokens'])]

##### 기타 전처리 (유효한 행만 남긱기)

In [ ]:
# 특정 형태소가 들어간 행만 남기기
query_list = ['울다','훌쩍이다'] # OR 조건
# df['tokens'].map(lambda x:query in x)
df[df['tokens'].map(lambda x:any(query in x for query in query_list))]

,idx,title,paragraph,dialogue,tokens
23,24,혈의 누/현대어 해석,밤은 깊어 사람의 자취도 없고 사면에서 닭은 홰를 치며 울고 개는 여염집 평대문 개...,0,"[깊다, 사람, 자취, 사면, 치다, 울다, 여염집, 평대문, 개구멍, 주둥이, 내..."
24,25,혈의 누/현대어 해석,"“개야, 너 혼자 집을 지키고 있구나. 우리가 피란 갈 때에 너를 부엌에 가두고 나...",1,"[혼자, 지키다, 우리, 피란, 가다, 부엌, 가두다, 나오다, 어디, 나오다, 같..."
49,50,혈의 누/현대어 해석,두 날개 탁탁 치며 꼬끼요 우는 소리는 첫닭이 분명한데 이 밤 새우기는 참 어렵도다...,0,"[날개, 치다, 꼬끼, 울다, 소리, 첫닭, 분명, 새우다, 어렵다, 그렇게, 적적..."
72,73,혈의 누/현대어 해석,평양의 난리 소문이 다른 사람 듣기에는 이웃집에 초상났다는 소문과 같이 심상히 들리...,0,"[평양, 난리, 소문, 다른, 사람, 듣다, 이웃집, 초상, 나다, 소문, 같이, ..."
87,88,혈의 누/현대어 해석,뒤떨어졌던 고장팔의 모가 들어 달아오면서 덩달아 운다.,0,"[뒤떨어지다, 고장, 들다, 달다, 오다, 덩달다, 울다]"
...,...,...,...,...,...
3156,1158,치악산,"정월 초하룻날 밤부터는 옥단이가 안방 상직잠을 자는 터이라, 고두쇠가 빈 행랑방에서...",0,"[정월, 초하룻날, 옥단이, 안방, 상직, 자다, 고두쇠, 비다, 행랑, 심심, 울..."
3157,1159,치악산,"“조 방정맞은 귀신, 제가 울면 누구를 어찌할 터인가, 제가 사람을 잡아 갈 수 있...",1,"[방정맞다, 귀신, 울다, 누구, 어찌, 사람, 잡다, 가다, 벌써, 오다, 잡아가..."
3168,1170,치악산,(고)“본래 아씨를 치악산에 버리고 오기는 소인이 한 일이올시다. 소인이 이 길로 ...,1,"[본래, 아씨, 치악산, 버리다, 오다, 소인, 소인, 원귀, 울다, 혼자, 나가다..."
3175,1177,치악산,"시골구석의 무식한 사람들이 귀신을 어찌 몹시 믿던지, 고두쇠란 놈은 홍참의 며느리 ...",0,"[시골, 구석, 무식, 사람, 귀신, 어찌, 몹시, 믿다, 고두쇠, 홍참, 며느리,..."


In [ ]:
df['tokens'].map(lambda x:len(x))

In [ ]:
df['tokens'].map(lambda x:len(x)).describe()

In [ ]:
df['tokens'].map(lambda x:len(x)).hist()

In [ ]:
# token 개수가 N개 이상인 행만 살리기
df = df[df['tokens'].map(lambda x:len(x)>=3)] #3개 이상
df

In [ ]:
# paragraph 열 중복행이 있으면 제거 
df = df.drop_duplicates(subset=['paragraph'])

In [ ]:
# reset index
df = df.reset_index(drop=True)

### 2. 키워드 추출 / 단어 네트워크

##### Term Frequency (단순 빈도수)

In [ ]:
tf_vectorizer = CountVectorizer(analyzer='word',
                             lowercase=False,
                             tokenizer=None,
                             preprocessor=None,
                             min_df=100, # 최소 100개 문서에서 등장해야 함
                             ngram_range=(1,2) #bigram까지
                             )

In [ ]:
tf_vector = tf_vectorizer.fit_transform(df['tokens'].astype(str))

In [ ]:
# 빈도수 내림차순으로 정렬
tf_scores = tf_vector.toarray().sum(axis=0)
tf_idx = np.argsort(-tf_scores)
tf_scores = tf_scores[tf_idx]
tf_vocab = np.array(tf_vectorizer.get_feature_names())[tf_idx]

In [ ]:
# 상위 50개 단어
print(list(zip(tf_vocab, tf_scores))[:50])

In [ ]:
# 워드클라이드 (참고용)
keywords = dict(zip(tf_vocab, tf_scores))

# wordcloud = wordcloud.generate_from_text(texts)
wordcloud = wordcloud.generate_from_frequencies(keywords)
wordcloud = WordCloud(
    font_path = font_path,
    width = 800,
    height = 800
)
wordcloud = wordcloud.generate_from_frequencies(keywords)
array = wordcloud.to_array()
fig = plt.figure(figsize=(5, 5))
plt.imshow(array, interpolation="bilinear")
plt.show()

##### TF-IDF(빈도수 * 역문서 빈도수)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(analyzer='word',
                             lowercase=False,
                             tokenizer=None,
                             preprocessor=None,
                             min_df=100, 
                             ngram_range=(1,2), #bigram 
                             smooth_idf=True)

In [ ]:
tfidf_vector = tfidf_vectorizer.fit_transform(kim['token'].astype(str))

In [ ]:
tfidf_scores = tfidf_vector.toarray().sum(axis=0)
tfidf_idx = np.argsort(-tfidf_scores)
tfidf_scores = tfidf_scores[tfidf_idx]
tfidf_vocab = np.array(tfidf_vectorizer.get_feature_names())[tfidf_idx]

In [ ]:
print(list(zip(tfidf_vocab, tfidf_scores))[:50]) #상위 50개 단어

##### 단어 빈도수 테이블 정리

In [ ]:
###TF, TF-IDF 단어 테이블 정리###
list(zip(tf_vocab, tf_scores,tfidf_vocab,tfidf_scores))[:100] #상위 100개
tf_tfidf_vocab = pd.DataFrame(list(zip(tf_vocab, tf_scores,tfidf_vocab,tfidf_scores)),
                              columns=['TF 단어','TF','TFIDF 단어','TFIDF'])

In [ ]:
tf_tfidf_vocab

In [ ]:
tf_tfidf_vocab.to_excel('이인직 키워드 빈도.xlsx') # 엑셀파일로 저장

##### 코사인 유사도 기반의 단어-단어 행렬

In [ ]:
tfidf_term_term_mat = cosine_similarity(tfidf_vector.T)
tfidf_term_term_mat = pd.DataFrame(tfidf_term_term_mat,index=tfidf_vectorizer.vocabulary_,columns=tfidf_vectorizer.vocabulary_)

In [ ]:
tfidf_term_term_mat

In [ ]:
# tf-idf 기준 상위 100개 단어만# tf-idf 기준 상위 100개 단어만
tfidf_term_term_mat_100 = tfidf_term_term_mat[tfidf_term_term_mat.keys().isin(tfidf_vocab[:100])]
tfidf_term_term_mat_100 = tfidf_term_term_mat_100[tfidf_term_term_mat_100.columns.intersection(tfidf_vocab[:100])]
tfidf_term_term_mat_100

In [ ]:
# csv로 저장
tfidf_term_term_mat_100.iloc[:100,:100].to_csv('단어 매트릭스.csv')